In [1]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from find_screen_utils import *
from find_object_utils import *
from __future__ import division

#Add Gaze Position to Video

In [11]:
gaze = pd.read_csv('images/feature_matching/tobii_data.csv')
vid_sync = pd.read_csv('images/feature_matching/tobii_data_video_sync.csv')

In [13]:
gaze['Vid_Time'] = (gaze['index'] - vid_sync.iloc[0][0])/1000000

In [14]:
gaze = gaze[gaze.Vid_Time > 0] # only start tracking eyes once video starts

In [15]:
infile = 'images/feature_matching/vid_tobii.mp4'
outfile = 'images/feature_matching/vid_tobii_gaze.m4v'

In [16]:
vid = cv2.VideoCapture(infile)

size = (1920, 1080)
fourcc = cv2.cv.CV_FOURCC(*'mp4v')
out = cv2.VideoWriter()
out.open(outfile, fourcc, vid.get(cv2.cv.CV_CAP_PROP_FPS)*2, size, True)
# note doubled framerate to include all eye tracking coordinates

i = 0
gaze_val = gaze['gaze position validity'].values
gaze_x = gaze['gaze position x'].values
gaze_y = gaze['gaze position y'].values

while(vid.isOpened()):
    ret, frame = vid.read()
    if ret:
        for _ in range(2):
            frame_cp = frame.copy()
            if gaze_val[i] == 0:
                gp_x = gaze_x[i]
                gp_y = gaze_y[i]
                cv2.circle(frame_cp, (int(1920*gp_x), int(1080*gp_y)), 8, [255,0,0], -2)
            out.write(frame_cp)
            i += 1
    else:
        break


vid.release()
out.release()

#Extract Screen From Gaze Video

In [2]:
infile = 'images/tobii/border/video/video_disp_gaze.m4v'
outfile = 'images/tobii/border/video/video_disp_gaze_screen2.m4v'

In [3]:
vid = cv2.VideoCapture(infile)

size = (1280, 720)
fourcc = cv2.cv.CV_FOURCC(*'mp4v')
out = cv2.VideoWriter()
out.open(outfile, fourcc, vid.get(cv2.cv.CV_CAP_PROP_FPS), size, True)

last_good = np.zeros((720, 1280, 3), dtype=np.uint8)  # represents the last good frame

i = 1
tot = vid.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT)
while(vid.isOpened()):
    ret, frame = vid.read()
    if ret:
        proc_frame = process_frame_lines(frame, 'light')
        
        if proc_frame is None:  # if screen isn't found, save image of frame
            out.write(last_good)  # write the last good frame to the video
            cv2.imwrite('images/tobii/border/video/dropped/frame%i.bmp'%(i), frame)
            pass
        else:  # frame is good
            last_good = proc_frame
            out.write(proc_frame)
        i += 1
        if i%50 == 0:
            print '%0.2f%%'%((i/tot)*100),
    else:
        break


vid.release()
out.release()

4.16%
8.31%
12.47%
16.63%
20.78%
24.94%
29.09%
33.25%
37.41%
41.56%
45.72%
49.88%
54.03%
58.19%
62.34%
66.50%
70.66%
74.81%
78.97%
83.13%
87.28%
91.44%
95.59%
99.75%


#Plot Gaze Onto Still Image With Object Detection

In [8]:
gaze = pd.read_csv('images/feature_matching/tobii_data.csv')
vid_sync = pd.read_csv('images/feature_matching/tobii_data_video_sync.csv')

gaze['Vid_Time'] = (gaze['index'] - vid_sync.iloc[0][0])/1000000
gaze = gaze[gaze.Vid_Time > 0] # only start tracking eyes once video starts

infile = 'images/feature_matching/vid_tobii.mp4'
outfile = 'images/feature_matching/vid_still_gaze.m4v'

In [9]:
vid = cv2.VideoCapture(infile)
match = cv2.imread('images/feature_matching/match.jpeg')

size = (match.shape[1], match.shape[0])
fourcc = cv2.cv.CV_FOURCC(*'mp4v')
out = cv2.VideoWriter()
out.open(outfile, fourcc, vid.get(cv2.cv.CV_CAP_PROP_FPS)*2, size, True)
# note doubled framerate to include all eye tracking coordinates

i = 1
tot = vid.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT)*2

gaze_val = gaze['gaze position validity'].values
gaze_x = gaze['gaze position x'].values
gaze_y = gaze['gaze position y'].values

gray_match = cv2.cvtColor(match, cv2.COLOR_BGR2GRAY)

while(vid.isOpened()):
    ret, frame = vid.read()
    if ret:
        match_cp = match.copy()
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        M = object_find(gray_match, gray_frame, 10)
        for _ in range(2):
            if gaze_val[i] == 0 and M is not None and i>250: # i>250 so we don't plot messy period
                org_pos = np.array((1920*gaze_x[i], 1080*gaze_y[i])).reshape(-1,1,2)
                trans_pos = cv2.perspectiveTransform(org_pos, M)
                trans_pos = tuple(np.int32(trans_pos[0,0]))
                if (trans_pos[0] <= size[0] and trans_pos[0] >= 0 and
                    trans_pos[1] <= size[1] and trans_pos[1] >= 0):
                    cv2.circle(match_cp, trans_pos, 8, [255,0,0], -2) # draw blue circle on current frame
                    cv2.circle(match, trans_pos, 8, [0,255,0], 2) # draw green circle as trace
            out.write(match_cp)
            
            i += 1
        if i%50 == 0:
            print '%0.2f%%'%((i/tot)*100)
    else:
        break


vid.release()
out.release()